In [1]:
import numpy as np 
import pandas as pd
import pdb
import requests

from Classifications import *

In [2]:
df = pd.read_csv("Meteorite_Landings.csv")
df.columns = ['name', 'id', 'nametype', 'recclass', 'mass', 'fall', 'year',
       'reclat', 'reclong', 'GeoLocation']

## Drop mass NaNs & 0.0

In [3]:
df = df[(df.mass > 0.0) & (df.mass.isna() == False)]
df.shape

(45566, 10)

## Classifications

In [4]:
df = Classifications.classify_subclasses(df)

In [111]:
classification_df = df.copy()

## Country Coordinates (removes 0.0 0.0 corrdinates)

Excerpt those entries that are not the primary ones from antarctica (handle those below)

In [102]:
geo_df = df[(df.reclat.isna() == False) 
            & (df.reclong.isna() == False) 
            & (df.GeoLocation != '(0.0, 0.0)') # ocean placeholder
            & (df.GeoLocation != '(-71.5, 35.66667)') # antarctica 1
            & (df.GeoLocation != '(-84.0, 168.0)') # antarctice 2
           ].copy()


Handle main antarctic locations (add country/continent name and abreviation without API calls)

In [103]:
antarctic_df = df.loc[(df.GeoLocation == '(-71.5, 35.66667)') | (df.GeoLocation == '(-84.0, 168.0)')].copy()

country = np.array(["Antarctica" for x in range(antarctic_df.shape[0])])
country_abrv = np.array(["AQ" for x in range(antarctic_df.shape[0])])

antarctic_df['country'] = country
antarctic_df['country_abrv'] = country_abrv


Pull in the countries information for the coordinates and format for merge

In [104]:
countries_df = pd.read_csv("countries_master.csv")
countries_df.columns = ['id', 'reclat', 'reclong', 'country', 'country_abrv']
countries_df.drop(['reclat', 'reclong'], axis=1, inplace=True)

Combine country information with geo_df

In [105]:
geo_df = geo_df.merge(countries_df, on="id").copy()

In [106]:
geo_df.head()

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation,major_classification,subclass_category,country,country_abrv
0,Aachen,1,Valid,L5,21.0,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)",Chondrite,L,Germany,DE
1,Aarhus,2,Valid,H6,720.0,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)",Chondrite,H,Denmark,DK
2,Abee,6,Valid,EH4,107000.0,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)",Chondrite,E,Canada,CA
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)",Achondrite,Acapulcoite,Mexico,MX
4,Achiras,370,Valid,L6,780.0,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)",Chondrite,L,Argentina,AR


In [107]:
geo_df_final = pd.concat([geo_df, antarctic_df]).copy()


In [109]:
geo_df_final

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation,major_classification,subclass_category,country,country_abrv
0,Aachen,1,Valid,L5,21.00,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)",Chondrite,L,Germany,DE
1,Aarhus,2,Valid,H6,720.00,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)",Chondrite,H,Denmark,DK
2,Abee,6,Valid,EH4,107000.00,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)",Chondrite,E,Canada,CA
3,Acapulco,10,Valid,Acapulcoite,1914.00,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)",Achondrite,Acapulcoite,Mexico,MX
4,Achiras,370,Valid,L6,780.00,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)",Chondrite,L,Argentina,AR
5,Adhi Kot,379,Valid,EH4,4239.00,Fell,01/01/1919 12:00:00 AM,32.10000,71.80000,"(32.1, 71.8)",Chondrite,E,Pakistan,PK
6,Adzhi-Bogdo (stone),390,Valid,LL3-6,910.00,Fell,01/01/1949 12:00:00 AM,44.83333,95.16667,"(44.83333, 95.16667)",Chondrite,LL,Mongolia,MN
7,Agen,392,Valid,H5,30000.00,Fell,01/01/1814 12:00:00 AM,44.21667,0.61667,"(44.21667, 0.61667)",Chondrite,H,France,FR
8,Aguada,398,Valid,L6,1620.00,Fell,01/01/1930 12:00:00 AM,-31.60000,-65.23333,"(-31.6, -65.23333)",Chondrite,L,Argentina,AR
9,Aguila Blanca,417,Valid,L,1440.00,Fell,01/01/1920 12:00:00 AM,-30.86667,-64.55000,"(-30.86667, -64.55)",Chondrite,L,Argentina,AR
